## Jax IlluminaWGS ## 14Aug, 2020 join calling

## Notes
1. -mq  -q means quiet, no verbose at all
2. gsutil doesn't work on python3.8, which is installed in my plot env
3. global varibles and libraries
4. Remeber to update the WRKDIR for previously used notebooks
5. Remember do not add comments before %%bash
6. Remember to replace 'usuhsID' with 'Sample_Name' in sample_ids
7. make sure the json or tool dir are there for the next steps

In [1]:
#global notebook variables for both python and bash majic (by stdin arguments)
#WRKDIR = '/Users/mooreank/Desktop/Mark/UNHS'
WRKDIR = '/Users/pengl7/Downloads/WGS/Jax'
#PRJ_BUCKET = 'gs://nihnialngcbg-testing'
PRJ_BUCKET = 'gs://test-7cee72c0e768'
#PROJECT_ID = 'nih-nia-lng-cbg'
PROJECT_ID = 'singlecellseq'
#MYUSER = 'mooreank'
MYUSER = 'pengl7'
AUTOSOMES=[str(x) for x in list(range(1,23))]
SEXOMES=['X','Y']
CHROMOSOMES=AUTOSOMES + SEXOMES

# COHORT='UNHS'
# remember the folder sequence should also be the same in the local dir
COHORT = 'Jax'

# Notice that COHORT_BUCKET means the main data_folder path in the project bucket, instead of a bucket name
COHORT_BUCKET=f'{PRJ_BUCKET}/{COHORT}'

 -s option: This option allows the positional parameters to be set when invoking an interactive shell.

In [2]:
%%bash -s "$PRJ_BUCKET" "$COHORT"
PRJ_BUCKET=${1}
COHORT=${2}
echo ${PRJ_BUCKET}/${COHORT}

gs://test-7cee72c0e768/Jax


In [2]:
import pandas as pd
import numpy as np
import time
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt

#### Check the number of files inside the data folder

In [5]:
%%bash -s "$COHORT_BUCKET"

COHORT_BUCKET=${1}

gsutil ls ${COHORT_BUCKET}/ubams | wc -l
echo ${COHORT_BUCKET}/ubams
gsutil ls ${COHORT_BUCKET}/ubams


       8
gs://test-7cee72c0e768/Jax/ubams
gs://test-7cee72c0e768/Jax/ubams/KOLF2-ARID2-A2/
gs://test-7cee72c0e768/Jax/ubams/KUCG3-C1/
gs://test-7cee72c0e768/Jax/ubams/LNGPI1-C1/
gs://test-7cee72c0e768/Jax/ubams/NCRM1-C6/
gs://test-7cee72c0e768/Jax/ubams/NCRM5-C5/
gs://test-7cee72c0e768/Jax/ubams/NN0003932-C3/
gs://test-7cee72c0e768/Jax/ubams/NN0004297-C1/
gs://test-7cee72c0e768/Jax/ubams/PGP1-C2/


In [6]:
!gsutil ls gs://test-7cee72c0e768/Jax/ubams/NCRM1-C6/

gs://test-7cee72c0e768/Jax/ubams/NCRM1-C6/NCRM1-C6_GT20-02407_L003_200221.unmapped.bam


In [ ]:
def checkfortemplatefile(this_template_file):
    if not os.path.isfile(this_template_file):
        print('need ' + this_template_file)

# Part3:  Alignment: ubam to cram

In [6]:
cohort_df = pd.read_csv("../Jax/cohort_df.csv")
cohort_df.head()

,Sample_Name,usuhsID,Adaptor,S,LANE,READ,NUM,Flowcell,DATE
0,KOLF2-ARID2-A2,GT20-02408,AAGTCCAA-TATGAGTA,S303,L003,R1,1,20-cel-001,200221
1,KUCG3-C1,GT20-02406,TTATAACC-GATATCGA,S306,L003,R1,1,20-cel-001,200221
2,LNGPI1-C1,GT20-02411,CAAGCTAG-ACATAGCG,S302,L003,R1,1,20-cel-001,200221
3,NCRM1-C6,GT20-02407,GGACTTGG-CGCAGACG,S309,L003,R1,1,20-cel-001,200221
4,NCRM5-C5,GT20-02409,ATCCACTG-AGGTGCGT,S304,L003,R1,1,20-cel-001,200221


- Remember to replace the path of wdl_pipeine.yaml
- To follow Raph and Anni's notebook, put all the cram, bam and g.vcf files in the same folder
- Next time do not using inputs OUTPUTS=.... as the example in Raph's testing

In [ ]:
# Raph's testing
# Next time do not using inputs OUTPUTS=.... as the example in Raph's testing
gcloud alpha genomics pipelines run \
--project singlecellseq \
--pipeline-file gs://nihnialng-pd-wgs/tools/yamls/wdl_pipeline.yaml \
--zones us-central1-f --memory 7 \
--logging gs://test-7cee72c0e768/Jax/logs/KOLF2-ARID2-A2 \
--inputs-from-file WDL=/labseq/projects/dementia_wgs/tools/broad/PairedEndSingleSampleWf.gatk4.0.wdl \
--inputs-from-file WORKFLOW_INPUTS=/home/gibbsr/KOLF2-ARID2-A2.broadbam.hg38.updated.json \
--inputs-from-file WORKFLOW_OPTIONS=/home/gibbsr/generic.google-papi.options.json \
--inputs WORKSPACE=gs://test-7cee72c0e768/Jax/workspace/KOLF2-ARID2-A2 \
--inputs OUTPUTS=gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22 \
--labels=pipe=ubam_to_cram,sample=kolf2-arid2-a2,cohort=jax,user=gibbsr


In [11]:
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/KOLF2-ARID2-A22/

gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.aligned.duplicates_marked.recalibrated.bam.read_group_md5
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.alignment_summary_metrics
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.bai
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.bait_bias_detail_metrics
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.bait_bias_summary_metrics
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.bam
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.cram
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.cram.crai
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.cram.md5
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.cram.validation_report
gs://test-7cee72c0e768/Jax/hg38/align-wf/KOLF2-ARID2-A22/KOLF2-ARID2-A2.duplicate_metrics


In [ ]:
# my testing of single run
# Next time do not using inputs OUTPUTS=.... as the example in Raph's testing
gcloud alpha genomics pipelines run \
--project singlecellseq \
--pipeline-file gs://test-7cee72c0e768/Jax/tools/ggp_wdl_pipeline.yaml \
--zones us-central1-f --memory 7 \
--logging gs://test-7cee72c0e768/Jax/logs/crams/KUCG3-C1 \
--inputs-from-file WDL=/Users/pengl7/Downloads/Jax/tools/broad/PairedEndSingleSampleWf.gatk4.0.wdl \
--inputs-from-file WORKFLOW_INPUTS=/Users/pengl7/Downloads/Jax/jsons/broadbams/KUCG3-C1.broadbam.hg38.updated.json \
--inputs-from-file WORKFLOW_OPTIONS=/Users/pengl7/Downloads/Jax/jsons/PairedEndSingleSampleWf.gatk4.0.options.json \
--inputs WORKSPACE=gs://test-7cee72c0e768/Jax/workspace/KUCG3-C1 \
--inputs OUTPUTS=gs://test-7cee72c0e768/Jax/hg38/align-wf/KUCG3-C1 \
--labels=pipe=ubam_to_cram,sample=kucg3-c1,cohort=jax,user=pengl7


In [13]:
#check bam, cram, and gvcf counts which already output into the destination foldere
print('#These actually won\'t be that helpful because you are all using the same bucket path.\n')
print('#crams')
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/**.cram | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/**.crai | wc -l
print('#bams')
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/**.bam | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/**.bai | wc -l
print('#gvcfs')
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/*/**.g.vcf.gz | wc -l

#These actually won't be that helpful because you are all using the same bucket path.

#crams
       7
       7
#bams
       7
       7
#gvcfs
       7


### Cautions about NCRM1-C6:
- It failed at the last step due to missing of chunks.
- The 1st run exited as 1 after exporting all outputs
- The 2nd run stopped before finishing. It outputs all cram, bam, gvcf, but missing outputs of CheckPreValidation/ and ValidateCram/
- Check the bam file for the suspicious positions using IGV later

In [7]:
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/

gs://test-7cee72c0e768/Jax/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/02ee76b6-5159-472e-a3c3-9df43e25905d/
gs://test-7cee72c0e768/Jax/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/


In [13]:
#check bam, cram, and gvcf counts of the second run of NCRM1-C6
print('#These actually won\'t be that helpful because you are all using the same bucket path.\n')
print('#crams')
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/02ee76b6-5159-472e-a3c3-9df43e25905d/call-ConvertToCram/**.cram | wc -l
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/02ee76b6-5159-472e-a3c3-9df43e25905d/call-ConvertToCram/**.crai | wc -l
print('#bams')
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/02ee76b6-5159-472e-a3c3-9df43e25905d/call-GatherBamFiles/**.bam | wc -l
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/02ee76b6-5159-472e-a3c3-9df43e25905d/call-GatherBamFiles/**.bai | wc -l
print('#gvcfs')
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/02ee76b6-5159-472e-a3c3-9df43e25905d/call-MergeVCFs/**.g.vcf.gz | wc -l

#These actually won't be that helpful because you are all using the same bucket path.

#crams
       1
       1
#bams
       1
       1
#gvcfs
       1


In [14]:
#check bam, cram, and gvcf counts of the first run of NCRM1-C6
print('#These actually won\'t be that helpful because you are all using the same bucket path.\n')
print('#crams')
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-ConvertToCram/**.cram | wc -l
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-ConvertToCram/**.crai | wc -l
print('#bams')
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-GatherBamFiles/**.bam | wc -l
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-GatherBamFiles/**.bai | wc -l
print('#gvcfs')
!gsutil ls {COHORT_BUCKET}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-MergeVCFs/**.g.vcf.gz | wc -l

#These actually won't be that helpful because you are all using the same bucket path.

#crams
       1
       1
#bams
       1
       1
#gvcfs
       1


Copying the output files (cram and crai) of the first run of NCRM1-C6 which has validateCram where the second run doesn't have to my final dest path, although they may didn't pass some criteria. The left will be kept for trouble shooting.

In [16]:
print('#run these commands at terminal:\n')

print('#gvcfs')
print('gsutil -mq cp {}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-MergeVCFs/**.g.vcf.gz* \
{}/hg38/align-wf/NCRM1-C6/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#crams')
print('gsutil -mq cp {}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-ConvertToCram/**.cram* \
{}/hg38/align-wf/NCRM1-C6/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#bams')
print('gsutil -mq cp {}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-GatherBamFiles/**.bam \
{}/hg38/align-wf/NCRM1-C6/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#bais')
print('gsutil -mq cp {}/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-GatherBamFiles/**.bai \
{}/hg38/align-wf/NCRM1-C6/'.format(COHORT_BUCKET,COHORT_BUCKET))

#run these commands at terminal:

#gvcfs
gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-MergeVCFs/**.g.vcf.gz* gs://test-7cee72c0e768/Jax/hg38/align-wf/NCRM1-C6/

#crams
gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-ConvertToCram/**.cram* gs://test-7cee72c0e768/Jax/hg38/align-wf/NCRM1-C6/

#bams
gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-GatherBamFiles/**.bam gs://test-7cee72c0e768/Jax/hg38/align-wf/NCRM1-C6/

#bais
gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/NCRM1-C6/PairedEndSingleSampleWorkflow/4217bcad-b3f6-45bf-84b1-b4896ea4ca57/call-GatherBamFiles/**.bai gs://test-7cee72c0e768/Jax/hg38/align-wf/NCRM1-C6/


Put all the cram, bam and g.vcf files in the same folder
Remember: do not use inputs OUTPUTS=.... next time

In [21]:
print('#run these commands at terminal:\n')

print('#gvcfs')
print('gsutil -mq cp {}/hg38/align-wf/*/**.g.vcf.gz* \
{}/hg38/gvcfs/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#crams')
print('gsutil -mq cp {}/hg38/align-wf/*/**.cram* \
{}/hg38/crams/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\n#bams')
print('gsutil -mq cp {}/hg38/align-wf/*/**.bam \
{}/hg38/bams/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('gsutil -mq cp {}/hg38/align-wf/*/**.bai \
{}/hg38/bams/'.format(COHORT_BUCKET,COHORT_BUCKET))

#run these commands at terminal:

#gvcfs
gsutil -mq cp gs://test-7cee72c0e768/Jax/hg38/align-wf/*/**.g.vcf.gz* gs://test-7cee72c0e768/Jax/hg38/gvcfs/

#crams
gsutil -mq cp gs://test-7cee72c0e768/Jax/hg38/align-wf/*/**.cram* gs://test-7cee72c0e768/Jax/hg38/crams/

#bams
gsutil -mq cp gs://test-7cee72c0e768/Jax/hg38/align-wf/*/**.bam gs://test-7cee72c0e768/Jax/hg38/bams/
gsutil -mq cp gs://test-7cee72c0e768/Jax/hg38/align-wf/*/**.bai gs://test-7cee72c0e768/Jax/hg38/bams/


In [22]:
#confirm counts of the primary output files at final dest path
#check bam, cram, and gvcf counts at final path
print('#crams')
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.cram | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.crai | wc -l
print('#bams')
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bam | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bai | wc -l
print('#gvcfs')
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.g.vcf.gz | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.g.vcf.gz.tbi | wc -l

#crams
       8
       8
#bams
       8
       8
#gvcfs
       8
       8


#### Check that all expected files are there by looking at sample IDs

In [24]:
%%bash -s "$WRKDIR" "$COHORT_BUCKET" "$COHORT"
#get a list of files that were successfully created
WRKDIR=${1}
COHORT_BUCKET=${2}
COHORT=${3}

gsutil -mq ls ${COHORT_BUCKET}/hg38/crams/*.cram > ${WRKDIR}/${COHORT}.found.files

sed -i -e s"/gs:\/\/test-7cee72c0e768\/${COHORT}\/hg38\/crams\///"g ${WRKDIR}/${COHORT}.found.files
sed -i -e s"/\///"g ${WRKDIR}/${COHORT}.found.files
sed -i -e s"/\.cram//"g ${WRKDIR}/${COHORT}.found.files

less ${WRKDIR}/${COHORT}.found.files | wc -l

8


In [27]:
%ls -lh /Users/pengl7/Downloads/WGS/Jax/*list*

-rwxrwxrwx@ 1 pengl7  1360859114   964B Aug 12 16:29 /Users/pengl7/Downloads/WGS/Jax/Jax.fastq.listing.txt*
-rwxrwxrwx@ 1 pengl7  1360859114   964B Aug 12 16:29 /Users/pengl7/Downloads/WGS/Jax/Jax.fastq.listing.txt-e*
-rwxrwxrwx@ 1 pengl7  1360859114     0B Aug  3 13:41 /Users/pengl7/Downloads/WGS/Jax/Jax.missing.samples.list*
-rwxrwxrwx@ 1 pengl7  1360859114    86B Aug  3 13:41 /Users/pengl7/Downloads/WGS/Jax/Jax.samples.list*


In [29]:
#check for any missing expected bams
expected_file = '{}/{}.samples.list'.format(WRKDIR, COHORT)
observed_file = '{}/{}.found.files'.format(WRKDIR, COHORT) 
missing_file = '{}/{}.missing.samples.list'.format(WRKDIR, COHORT)

expected = pd.read_csv(expected_file,header=None)
observed = pd.read_csv(observed_file,header=None)

print(expected.shape)
print(observed.shape)

print(len(set(expected[0]) - set(observed[0])))

missing = expected.loc[~expected[0].isin(observed[0])]
print(missing.shape)
missing.head()

#save the missing list
missing.to_csv(missing_file,header=None,index=None)

(8, 1)
(8, 1)
0
(0, 1)


In [31]:
# move the summary metric files that may be of interest to keep
# this block is for the 7 successful samples and one failed 
print('#run these commands at terminal:\n')
print('gsutil -mq cp {}/workspace/*/PairedEndSingleSampleWorkflow/*/call-ValidateCram/**.cram.validation_report ' \
'{}/hg38/align-wf/call-ValidateCram/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/workspace/*/PairedEndSingleSampleWorkflow/*/call-CheckContamination/**.preBqsr.selfSM ' \
'{}/hg38/align-wf/call-CheckContamination/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/workspace/*/PairedEndSingleSampleWorkflow/*/call-GatherBqsrReports/**.recal_data.csv ' \
'{}/hg38/align-wf/call-GatherBqsrReports/'.format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/workspace/*/PairedEndSingleSampleWorkflow/**_metrics {}/hg38/align-wf/metrics/'.\
      format(COHORT_BUCKET,COHORT_BUCKET))
print('\ngsutil -mq cp {}/workspace/*/PairedEndSingleSampleWorkflow/**.pdf {}/hg38/align-wf/metrics/'.\
      format(COHORT_BUCKET,COHORT_BUCKET))

#run these commands at terminal:

gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/*/PairedEndSingleSampleWorkflow/*/call-ValidateCram/**.cram.validation_report gs://test-7cee72c0e768/Jax/hg38/align-wf/call-ValidateCram/

gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/*/PairedEndSingleSampleWorkflow/*/call-CheckContamination/**.preBqsr.selfSM gs://test-7cee72c0e768/Jax/hg38/align-wf/call-CheckContamination/

gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/*/PairedEndSingleSampleWorkflow/*/call-GatherBqsrReports/**.recal_data.csv gs://test-7cee72c0e768/Jax/hg38/align-wf/call-GatherBqsrReports/

gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/*/PairedEndSingleSampleWorkflow/**_metrics gs://test-7cee72c0e768/Jax/hg38/align-wf/metrics/

gsutil -mq cp gs://test-7cee72c0e768/Jax/workspace/*/PairedEndSingleSampleWorkflow/**.pdf gs://test-7cee72c0e768/Jax/hg38/align-wf/metrics/


In [33]:
#check file counts at destination for primary and other metrics output files
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.vcf.gz | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/gvcfs/*.vcf.gz.tbi | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.cram | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/crams/*.crai | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bam | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/bams/*.bai | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/call-ValidateCram/*.cram.validation_report | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/call-CheckContamination/*.preBqsr.selfSM | wc -l
!gsutil ls {COHORT_BUCKET}/hg38/align-wf/call-GatherBqsrReports/*.recal_data.csv | wc -l

8
       8
       8
       8
       8
       8
       8
       8
       8


### check the cram validation reports

In [35]:
# pull down the validation reports and check for erros
# for NCRM1-C6 first run, it passed the checkCram
!mkdir -p {WRKDIR}/temp

!gsutil -mq cp {COHORT_BUCKET}/hg38/align-wf/call-ValidateCram/*.cram.validation_report {WRKDIR}/temp/

!ls {WRKDIR}/temp/*.validation_report | wc -l
!less {WRKDIR}/temp/*.validation_report | grep "No errors found" | wc -l

#clean up the temp validation reports
!rm {WRKDIR}/temp/*.validation_report

8
       8


## This step can be executated later

### Do some additional cleanup, including deletion of the ubams

In [ ]:
#if successfully, clean up log files and workspace path
print('gsutil -mq rm -r {}/logs/crams'.format(COHORT_BUCKET))
print('gsutil -mq rm -r {}/workspace'.format(COHORT_BUCKET))


In [ ]:
#define format function the gcp cmd to delete additional per sample un-needed files
def formatgcpcmd(this_sample,chrt_bucket):
    this_cmd = 'gsutil -mq rm -r {COHORT_BUCKET}/hg38/align-wf/{SAMPLE}/PairedEndSingleSampleWorkflow'
    return(this_cmd.format(COHORT_BUCKET=chrt_bucket,SAMPLE=this_sample))

#here reloading sample_ids
cohort_file_list = '{}/{}.samples.list'.format(WRKDIR,COHORT)
sample_ids = pd.read_csv(cohort_file_list,header=None).to_numpy()
sample_ids = sample_ids.reshape(len(sample_ids))

#iterate over samples formatting the cmds
cmds = [formatgcpcmd(sample_id,COHORT_BUCKET) for sample_id in sample_ids]

temp_script_file = '{}/{}.delete_other_files.sh'.format(WRKDIR,COHORT.lower())

with open(temp_script_file, 'w') as file_handler:
        for this_cmd in cmds:
            file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO DELETE AGAIN\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/{}.delete_other_files.log &'.format(WRKDIR,COHORT.lower()))

In [ ]:
##didnt do this yet

#define format function the ggp cmd to delete ubams
def formatgcpcmd(this_sample,chrt_bucket):
    this_cmd = 'gsutil -mq rm -r {COHORT_BUCKET}/ubams/{SAMPLE}'
    return(this_cmd.format(COHORT_BUCKET=chrt_bucket,SAMPLE=this_sample))

#iterate over samples formatting the cmds
cmds = [formatgcpcmd(sample_id,COHORT_BUCKET) for sample_id in sample_ids]

temp_script_file = '{}/{}.delete_ubams.sh'.format(WRKDIR,COHORT.lower())

with open(temp_script_file, 'w') as file_handler:
        for this_cmd in cmds:
            file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO DELETE AGAIN\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/{}.delete_ubams.log &'.format(WRKDIR,COHORT.lower()))

## The above hasn't been executed yet!

### check for any contaminated samples
pull down seq contamination check info

In [3]:
#pull down the verifybamid check contamination files 
!mkdir -p {WRKDIR}/seqqc

!gsutil -mq cp {COHORT_BUCKET}/hg38/align-wf/call-CheckContamination/*.preBqsr.selfSM {WRKDIR}/seqqc/
!ls {WRKDIR}/seqqc/*.preBqsr.selfSM | wc -l



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

       8


In [4]:
# check contamination info and export to the file of .contam.metrics.txt
sample_file = f'{WRKDIR}/{COHORT}.samples.list'

samples_list= []
with open(sample_file, 'r') as fi:
    for line in fi:
        line = line.strip()
        samples_list.append(str(line))
        

with open(f"{WRKDIR}/{COHORT}.contam.metrics.txt", "w") as text_file:
    text_file.write("id avgdp freemix \n")
    
for sample in samples_list:
    cmd = f"""awk 'NR==2{{print $1,$6,$7}}' {WRKDIR}/seqqc/{sample}.preBqsr.selfSM >> {WRKDIR}/{COHORT}.contam.metrics.txt"""
    !{cmd}

Found 7 out of 8 samples have the avgdp less than the mininal depth
- checked the batch of our Psomagen data and found all of them are much higher than the minial depth (27), for example, KOLF2-ARID2 is 41 and NCRM1-C6 is 50.

In [39]:
#check the contamination and seq coverage rates
contam_metrics_file = '{}/{}.contam.metrics.txt'.format(WRKDIR,COHORT)

WARN_FREEMIX = 0.02
MAX_FREEMIX = 0.049
MIN_DEPTH = 27

metrics_df = pd.read_csv(contam_metrics_file,sep='\s+')
print(metrics_df.shape)
metrics_df.head()

maybe_contamin_df = metrics_df.loc[(metrics_df['freemix'] <= MAX_FREEMIX) & \
                                   (metrics_df['freemix'] > WARN_FREEMIX)]
print('number of samples maybe contaminated with freemix > {} and < {} is {}'\
      .format(WARN_FREEMIX,MAX_FREEMIX,maybe_contamin_df.shape[0]))
contam_maybe_file = '{}/{}.contamination.possible.samples.txt'.format(WRKDIR,COHORT)
maybe_contamin_df.to_csv(contam_maybe_file,index=False,sep='\t')

contaminated_df = metrics_df.loc[metrics_df['freemix'] > MAX_FREEMIX]
print('number of samples with freemix > {} is {}'.format(MAX_FREEMIX,contaminated_df.shape[0]))
contam_problems_file = '{}/{}.contaminated.samples.txt'.format(WRKDIR,COHORT)
contaminated_df.to_csv(contam_problems_file,index=False,sep='\t')

low_cov_df = metrics_df.loc[metrics_df['avgdp'] < MIN_DEPTH]
print('number of samples with avgdp < {} is {}'.format(MIN_DEPTH,low_cov_df.shape[0]))
low_coverage_file = '{}/{}.low_coverage.samples.txt'.format(WRKDIR,COHORT)
low_cov_df.to_csv(low_coverage_file,index=False,sep='\t')

(8, 3)
number of samples maybe contaminated with freemix > 0.02 and < 0.049 is 0
number of samples with freemix > 0.049 is 0
number of samples with avgdp < 27 is 7


### Since the number of samples contaminated is 0, we can go ahead to joint genotyping
### Generate two jsons files need for joint genotyping
1. specify the options for join genotyping using the generic.googlee-papi.options.json template
2. specify the gvcf map text, callset name, sample name map using the joint-discovery-gatk4.hg38.wgs.inputs.json template

- updated the old joint-discovery-gatk4.hg38.wgs.inputs.json with the new broad bucket name
- I changed the bucket name in place, becasue no need to keep the old one

In [21]:
#update some of the json configs for broad joint calling
#generate the extra json files for ubams to crams; ie labels and options for alignment wf

# generic_options_template = f'{WRKDIR}/jsons/generic.google-papi.options.json'
# wdl_input_template = f'{WRKDIR}/jsons/JointGenotyping.hg38.wgs.inputs.json'

#json_labels_outfile_name = '{}/jsons/{}.JointGenotyping.labels.json'.format(WRKDIR,COHORT)
# json_options_outfile_name = f'{WRKDIR}/jsons/{COHORT}.JointGenotyping.options.json'
# json_broad_outfile_name = f'{WRKDIR}/jsons/{COHORT}.JointGenotyping.hg38.wgs.inputs.json'  

generic_options_template = f'{WRKDIR}/jsons/generic.google-papi.options.json'
wdl_input_template = f'{WRKDIR}/jsons/joint-discovery-gatk4.hg38.wgs.inputs.json'

#json_labels_outfile_name = '{}/jsons/{}.jdgatk4.labels.json'.format(WRKDIR,COHORT)
json_options_outfile_name = f'{WRKDIR}/jsons/{COHORT}.jdgatk4.options.json'
json_broad_outfile_name = f'{WRKDIR}/jsons/{COHORT}.jdgatk4.hg38.wgs.inputs.json'

#format and write the label json
# label_data = {}
# label_data['workflow'] = 'jdgatk4'
# label_data['cohort'] = COHORT.lower()
# label_data['user'] = MYUSER.lower()

# with open(json_labels_outfile_name,'w') as json_outfile:
#     json.dump(label_data,json_outfile,sort_keys=True,indent=4)   

#format and write the options json
with open(generic_options_template) as json_file:  
    options_data = json.load(json_file)
    
    options_data['read_from_cache'] = True
    options_data['write_to_cache'] = True
    options_data['workflow_failure_mode'] = 'ContinueWhilePossible'
    options_data['system.input-read-limits.lines'] = 640000    
    options_data['final_workflow_outputs_dir'] = '{}/hg38/joint_calling'.format(COHORT_BUCKET)
    options_data['final_workflow_log_dir'] = '{}/logs/joint_calling'.format(COHORT_BUCKET)
    options_data['final_call_logs_dir'] = '{}/logs/joint_calling'.format(COHORT_BUCKET)

    with open(json_options_outfile_name,'w') as json_outfile:
        json.dump(options_data,json_outfile,sort_keys=True,indent=4)   

#format and write the broad json
with open(wdl_input_template) as json_file:  
    broad_data = json.load(json_file)
    
    sample_gvcfs_path = '{}/{}.sample.gvcfs.map.txt'.format(COHORT_BUCKET,COHORT)
    broad_data['JointGenotyping.callset_name'] = COHORT
    broad_data['JointGenotyping.sample_name_map'] = sample_gvcfs_path

    with open(json_broad_outfile_name,'w') as json_outfile:
        json.dump(broad_data,json_outfile,sort_keys=False,indent=4)   

In [62]:
# create the gvcfs sample map file used as json argument above JointGenotyping.sample_name_map
#instead of just looping expected ids do so from found list in previous cell, allow to move/remove fails

found_file_list = '{}/{}.found.files'.format(WRKDIR,COHORT)
found_ids = pd.read_csv(found_file_list,header=None).to_numpy()
# found_ids = sample_ids.reshape(len(found_ids))
found_ids = found_ids.reshape(len(found_ids))

gvcfs_map_file = '{}/{}.sample.gvcfs.map.txt'.format(WRKDIR,COHORT)

with open(gvcfs_map_file, 'w') as file_handler:
    for sample_id in found_ids:
        file_handler.write("{}\t{}/hg38/gvcfs/{}.g.vcf.gz\n".\
                           format(sample_id,COHORT_BUCKET,sample_id))

#now copy the map file up to cloud bucket
!gsutil -mq cp {gvcfs_map_file} {COHORT_BUCKET}/

Note that the output diretory has been pre-defined here

In [17]:
#now run the joint calling job

# run_cmd = 'python {wrk_dir}/jsons/cromwell_client.py \
# --wdl {wrk_dir}/tools/joint-discovery-gatk4.wdl \
# --workflow-inputs {wrk_dir}/jsons/{this_cohort}.jdgatk4.hg38.wgs.inputs.json \
# --workflow-options {wrk_dir}/jsons/{this_cohort}.jdgatk4.options.json \
# --workflow-labels {wrk_dir}/jsons/{this_cohort}.jdgatk4.labels.json'.\
# format(wrk_dir=WRKDIR, this_cohort=COHORT)

# print('#run these commands at terminal:\n')

# print('{} > {}/{}.jdgatk4.jobid'.format(run_cmd, WRKDIR, COHORTBUILD))

cohort_bucket = f'{PRJ_BUCKET}/{COHORT}'

def formatgcpcmd(chrt_bucket):
    this_cmd = 'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {COHORT_BUCKET}/tools/ggp_wdl_pipeline.yaml \
--zones us-central1-f \
--memory 7 \
--logging {COHORT_BUCKET}/logs/joint-discovery/ \
--inputs-from-file WDL={WRKDIR}/tools/broad/joint-discovery-gatk4.wdl \
--inputs-from-file WORKFLOW_INPUTS={WRKDIR}/jsons/{COHORT}.jdgatk4.hg38.wgs.inputs.json \
--inputs-from-file WORKFLOW_OPTIONS={WRKDIR}/jsons/{COHORT}.jdgatk4.options.json \
--inputs WORKSPACE={COHORT_BUCKET}/workspace/ \
--inputs OUTPUTS={COHORT_BUCKET}/hg38/joint-discovery/ \
--labels=pipe=jointdiscovery,cohort={LCCOHORT},user={MYUSER}'
    return(this_cmd.format(PROJECT_ID=PROJECT_ID,PRJ_BUCKET=PRJ_BUCKET,COHORT=COHORT,\
                         COHORT_BUCKET=chrt_bucket,WRKDIR=WRKDIR,LCCOHORT=COHORT.lower(),MYUSER=MYUSER))

# #iterate over samples formatting the cmds
# cohort_bucket = '{}/{}'.format(PRJ_BUCKET,COHORT)
# cmds = [formatgcpcmd(sample_id,cohort_bucket) for sample_id in sample_ids]

# temp_script_file = '{}/{}.run_JointGenotyping.sh'.format(WRKDIR,COHORT.lower())

final_cmd = formatgcpcmd(cohort_bucket)

# with open(temp_script_file, 'w') as file_handler:
#         for this_cmd in cmds:
#             file_handler.write("{}\n".format(this_cmd))
            
print('#run these commands at terminal:\n')
#print('chmod +x ' + final_cmd)
print('nohup ' +final_cmd + ' > {}/{}.joint-discovery.log &'.format(WRKDIR,COHORT.lower()))

#run these commands at terminal:

nohup gcloud alpha genomics pipelines run --project singlecellseq --pipeline-file gs://test-7cee72c0e768/Jax/tools/ggp_wdl_pipeline.yaml --zones us-central1-f --memory 7 --logging gs://test-7cee72c0e768/Jax/logs/joint-discovery/ --inputs-from-file WDL=/Users/pengl7/Downloads/WGS/Jax/tools/broad/joint-discovery-gatk4.wdl --inputs-from-file WORKFLOW_INPUTS=/Users/pengl7/Downloads/WGS/Jax/jsons/Jax.jdgatk4.hg38.wgs.inputs.json --inputs-from-file WORKFLOW_OPTIONS=/Users/pengl7/Downloads/WGS/Jax/jsons/Jax.jdgatk4.options.json --inputs WORKSPACE=gs://test-7cee72c0e768/Jax/workspace/ --inputs OUTPUTS=gs://test-7cee72c0e768/Jax/hg38/joint-discovery/ --labels=pipe=jointdiscovery,cohort=jax,user=pengl7 > /Users/pengl7/Downloads/WGS/Jax/jax.joint-discovery.log &


In [20]:
# see if result file is present, the below command didn't work becaue out output dir has been predefined in the pipeline run
#!gsutil ls -lh {COHORT_BUCKET}/hg38/joint_calling/*/{COHORTBUILD}.vcf.gz*

!gsutil ls -lh {COHORT_BUCKET}/hg38/joint-discovery/*.vcf.gz*


1.68 GiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.vcf.gz
  2.55 MiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.vcf.gz.tbi
TOTAL: 2 objects, 1801354867 bytes (1.68 GiB)


In [23]:
!gsutil ls {COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/8f1d94f8-efb4-4c0c-a4fe-35a6e86c5c38/

gs://test-7cee72c0e768/Jax/hg38/joint_calling/JointGenotyping/8f1d94f8-efb4-4c0c-a4fe-35a6e86c5c38/call-CollectMetricsOnFullVcf/
gs://test-7cee72c0e768/Jax/hg38/joint_calling/JointGenotyping/8f1d94f8-efb4-4c0c-a4fe-35a6e86c5c38/call-DynamicallyCombineIntervals/


This block hasn't been executed yet since we already output file directly to the dest

In [ ]:
#COHORTBUILD = 'UNHS'
#job was run twice to two sets of outputs, grab one and then do some cleanup
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO RUN AGAIN\n')
in_path = f'{COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/0822e1a2-f073-49fa-8dd3-bd9519dc7781'
out_path = f'{COHORT_BUCKET}/hg38/JointGenotyping/'
#print(f'gsutil -mq cp {in_path }/call-DynamicallyCombineIntervals/{COHORTBUILD}.vcf.gz* {out_path}')
print(f'gsutil -mq cp {in_path }/call-CollectMetricsOnFullVcf/* {out_path}')

In [31]:
!gsutil ls {COHORT_BUCKET}/logs/

gs://test-7cee72c0e768/Jax/logs/KOLF2-ARID2-A2/
gs://test-7cee72c0e768/Jax/logs/crams/
gs://test-7cee72c0e768/Jax/logs/joint-discovery/
gs://test-7cee72c0e768/Jax/logs/joint_calling/


In [32]:
# clean up
#delete logging, cromwell execution path, and detritus from output path
in_path = f'{COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/*'
print('#run these commands at terminal:\n')
print('#I\'VE ALREADY RUN THIS SO DO NOT NEED TO RUN AGAIN\n')

print(f'gsutil -mq rm {in_path}/call-FinalGatherVcf/{COHORT}.vcf.gz*')
print(f'gsutil -mq rm {in_path}/call-CollectMetricsOnFullVcf/*')
print(f'gsutil -mq rm {in_path}/call-DynamicallyCombineIntervals/*')
print(f'gsutil -mq rm -r {COHORT_BUCKET}/hg38/joint_calling/JointGenotyping/*')

print(f'gsutil -mq rm -r {COHORT_BUCKET}/logs/j*')
#print(f'gsutil -mq rm -r {COHORT_BUCKET}/cromwell-execution/JointGenotyping')

#run these commands at terminal:

#I'VE ALREADY RUN THIS SO DO NOT NEED TO RUN AGAIN

gsutil -mq rm gs://test-7cee72c0e768/Jax/hg38/joint_calling/JointGenotyping/*/call-FinalGatherVcf/Jax.vcf.gz*
gsutil -mq rm gs://test-7cee72c0e768/Jax/hg38/joint_calling/JointGenotyping/*/call-CollectMetricsOnFullVcf/*
gsutil -mq rm gs://test-7cee72c0e768/Jax/hg38/joint_calling/JointGenotyping/*/call-DynamicallyCombineIntervals/*
gsutil -mq rm -r gs://test-7cee72c0e768/Jax/hg38/joint_calling/JointGenotyping/*
gsutil -mq rm -r gs://test-7cee72c0e768/Jax/logs/j*


# Part4: subset vcfs to just PASSED variants

VQSR doesn't really work for the non-PAR sexomes and while you probably won't be doing analsys on sexome variation you will need at lest chrX to perform sex check QC¶

so submit another vcf subset but only for chrX without PASS filter

In [ ]:
print('#run these commands at terminal:\n')
#prep and run the VQSR subset job
print(f'cut -f 1 {WRKDIR}/{COHORT}.sample.gvcfs.map.txt > {WRKDIR}/{COHORT}.samples.list\n')
print(f'gsutil -mq cp {WRKDIR}/{COHORT}.samples.list {COHORT_BUCKET}/\n')

pipe_yaml = 'gs://nihnialngcbg-testing/tools/SubjectSubsetVCF.yaml'

filter_vqsr_cmd = f'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {pipe_yaml} \
--logging {COHORT_BUCKET}/logs/jointcall/filtervcf/ \
--inputs INPUTVCF={COHORT_BUCKET}/joint-discovery/{COHORTBUILD}.vcf.gz \
--inputs INPUTVCFINDEX={COHORT_BUCKET}/joint-discovery/{COHORTBUILD}.vcf.gz.tbi \
--inputs SUBJECTLIST={COHORT_BUCKET}/{COHORTBUILD}.samples.list \
--inputs EXCLUDECARROT=\" \" \
--inputs FILTERFLAGS=\"-f PASS --force-samples\" \
--inputs MINAC=\"1\" \
--outputs OUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.vcf.gz \
--outputs OUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.vcf.gz.tbi \
--labels=pipe=filtervcfs,cohort={COHORT.lower()},user={MYUSER}\n'

print(filter_vqsr_cmd)

chrx_vcf_cmd = f'gcloud alpha genomics pipelines run \
--project {PROJECT_ID} \
--pipeline-file {pipe_yaml} \
--logging {COHORT_BUCKET}/logs/jointcall/filtervcf/ \
--inputs INPUTVCF={COHORT_BUCKET}/joint-discovery/{COHORTBUILD}.vcf.gz \
--inputs INPUTVCFINDEX={COHORT_BUCKET}/joint-discovery/{COHORTBUILD}.vcf.gz.tbi \
--inputs SUBJECTLIST={COHORT_BUCKET}/{COHORTBUILD}.samples.list \
--inputs EXCLUDECARROT=\" \" \
--inputs FILTERFLAGS=\"--regions chrX --force-samples\" \
--inputs MINAC=\"1\" \
--outputs OUTVCF={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.vcf.gz \
--outputs OUTVCFINDEX={COHORT_BUCKET}/hg38/genotypes/{COHORTBUILD}.chrX.vcf.gz.tbi \
--labels=pipe=filtervcfs,cohort={COHORT.lower()},chromosome=chrx,user={MYUSER}\n'

print(chrx_vcf_cmd)

In [33]:
cmd2 = f'gcloud compute instances list --project {PROJECT_ID} | grep RUNNING | wc -l'
!{cmd2}

0


In [32]:

%%bash

OPID=EIjRoLrALhjnvYfhzqb_v7YBINHhrPbMCCoPcHJvZHVjdGlvblF1ZXVl
gcloud alpha genomics operations describe ${OPID} \
--format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2020-08-19T15:13:56.614257047Z'
  - description: pulling-image
    startTime: '2020-08-19T15:13:56.614330643Z'
  - description: localizing-files
    startTime: '2020-08-19T15:14:29.732031166Z'
  - description: running-docker
    startTime: '2020-08-19T15:14:29.732071540Z'
  - description: delocalizing-files
    startTime: '2020-08-19T23:12:10.280716580Z'
  - description: ok
    startTime: '2020-08-19T23:12:11.306508509Z'


### Check the example lifesciencee pipeline

In [ ]:
!export GATK_GOOGLE_DIR="${PWD}"/broad-prod-wgs-germline-snps-indels
!export GATK_OUTPUT_DIR=gs://transfer_27may/beta_test

In [ ]:
gcloud beta lifesciences pipelines run \
--project singlecellseq \
--pipeline-file gs://transfer_27may/Jax/tools/glsp_wdl_pipeline.yaml \
--location us-central1 \
--regions us-central1 \
--inputs-from-file WDL=${GATK_GOOGLE_DIR}/PairedEndSingleSampleWf.wdl,\
WORKFLOW_INPUTS=${GATK_GOOGLE_DIR}/PairedEndSingleSampleWf.hg38.inputs.json,\
WORKFLOW_OPTIONS=${GATK_GOOGLE_DIR}/PairedEndSingleSampleWf.options.json \
--env-vars WORKSPACE=${GATK_OUTPUT_DIR}/work,\
OUTPUTS=${GATK_OUTPUT_DIR}/output \
--logging ${GATK_OUTPUT_DIR}/logging/

In [ ]:
cmd2 = f'gcloud compute instances list --project {PROJECT_ID} | grep RUNNING | wc -l'
!{cmd2}

In [ ]:
%%bash

OPID=2439314507554778382
gcloud beta lifesciences operations describe --project singlecellseq ${OPID} \
--format='yaml(done, error, metadata.events)'